# EXPORTS JOBS POSTED FROM [web.byui.edu/StudentEmployment/’](web.byui.edu/StudentEmployment/) INTO A CSV FILE

In [1]:
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

## REQUEST DATA FROM API ENDPOINT

In [2]:
conn = http.client.HTTPSConnection("web.byui.edu")
 
conn.request("GET", "/studentemployment/api/jobs")

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

In [3]:
data_jobs['URL'] = data_jobs.jobID.apply(lambda x: f'https://web.byui.edu/StudentEmployment/job/{x}')

In [4]:
data_jobs['URL'].iloc[300]

'https://web.byui.edu/StudentEmployment/job/341519'

In [5]:
pd.options.display.max_columns = None
data_jobs

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,managerName,payRate,positionsAllocated,positionsAvailble,beginningDate,recruitingStartDate,requireResume,limitApplicants,limitNumber,applicants,jobQuestions,isOnline,allowOnline,jobMajors,startDate,endDate,approximateHoursPerWeek,workSchedule,URL
0,1588222,BCTR Late Night Custodian,Livestock / Night BCTR FMS,<p>Carpet cleaning and custodial work around c...,Cleaning campus buildings. Shift is 11:30 pm-2...,True,2022-09-07T05:13:39.5877101,Greg Carron,9.58,1000.0,978.0,2022-09-05T12:00:00,2021-01-01T01:00:00,True,True,10,[],[],False,False,[],NaN,NaN,NaN,NaN,https://web.byui.edu/StudentEmployment/job/158...
1,1360146,AGBUS147 OnlineTA,Campus Teaching Assistants,<p>A Teaching Assistant is needed for AGBUS 14...,A Teaching Assistant is needed for AGBUS 147: ...,True,2022-09-06T23:15:13.9391796,Esther Zonts,10.00,100.0,95.0,2022-09-14T12:00:00,2020-10-20T01:00:00,True,True,50,[],[],False,False,[],2022-06-27T08:00:00,2022-12-17T11:00:00,up to 5,Flexible,https://web.byui.edu/StudentEmployment/job/136...
2,1361237,AGBUS147 OnlineTA,Online Student,<p>A Teaching Assistant is needed for AGBUS 14...,A Teaching Assistant is needed for AGBUS 147: ...,True,2022-09-06T23:15:09.5556704,Esther Zonts,10.00,100.0,99.0,2022-09-14T12:00:00,2020-10-21T01:00:00,True,True,50,[],[],False,True,[],2022-06-26T21:00:00,2022-09-12T16:00:00,up to 5,Flexible,https://web.byui.edu/StudentEmployment/job/136...
3,2399931,Email Specialist,Main BSC Support Center,<div><div><p><span><span>As a BYU-Idaho Email ...,The BYU-Idaho Support Center is the primary su...,True,2022-09-06T22:31:55.4427994,Abigayle Ludlow,10.40,1000.0,959.0,2022-09-12T06:00:00,2021-08-10T01:00:00,True,False,0,[],[],False,False,[],2022-09-01T18:00:00,2022-09-09T18:00:00,NaN,1pm-5pm,https://web.byui.edu/StudentEmployment/job/239...
4,2182195,INTST100 OnlineTutor,Campus Student,<p><span> Online Tutor is needed for the INTST...,Online Tutor is needed for the INTST 100: Intr...,True,2022-09-06T21:31:54.3026567,Casey McDaniel,10.00,100.0,95.0,2022-09-12T23:00:00,2021-06-15T01:00:00,True,False,0,[],[],True,False,[],2022-08-09T21:00:00,2022-09-10T22:00:00,5-10 hours (must post 7 hours of appt openings...,flexible,https://web.byui.edu/StudentEmployment/job/218...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,96139,Student Project Manager,Online Curriculum Development,<p>The student project manager position undert...,Assist in managing the audiovisual section of ...,True,2022-07-29T15:12:11.4490012,Brian Carter,10.00,500.0,494.0,2022-09-09T12:00:00,2016-11-28T01:00:00,True,True,30,[],[],False,False,[],2022-07-29T12:00:00,2022-09-09T12:00:00,20,Varies,https://web.byui.edu/StudentEmployment/job/96139
338,3820655,Hard Floor Team Member,Building/Hard Floor FMS,"<p> Duties include cleaning, buffing, re-surfa...",This team member will work under the direction...,True,2022-07-25T20:17:12.8614333,Lauren Brisco,9.58,1000.0,989.0,NaN,2022-05-03T01:00:00,True,False,0,[],[],False,False,[],NaN,NaN,15,"Monday-Friday, 8PM-11PM",https://web.byui.edu/StudentEmployment/job/382...
339,32596,Chemistry Stockroom/TA/Grader,Chemistry,<p>This is a general application form for any ...,General Application form for all chemistry dep...,True,2022-06-09T20:08:58.2027174,Mike Wood,NaN,10000.0,10000.0,NaN,2016-11-11T01:00:00,True,False,0,[],[],False,False,"[{'jobMajorId': 212, 'jobId': 32596, 'majorId'...",2022-06-09T14:00:00,NaN,NaN,Varies,https://web.byui.edu/StudentEmployment/job/32596
340,341293,Security-Lock up Patrol,Public Safety,<p>Employees will assist with patrolling campu...,NaN,True,2022-02-18T23:30:52.6069815,Kevin Rhoades,NaN,200.0,135.0,NaN,2018-08-15T01:00:00,True,False,0,[],[],False,False,[],2021-12-05T07:00:00,NaN,NaN,NaN,https://web.byui.edu/StudentEmployment/job/341293


In [6]:
type(data)

bytes

## DISPLAY DATA RECEIVED

In [7]:
pd.options.display.max_columns = None

data_jobs

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,managerName,payRate,positionsAllocated,positionsAvailble,beginningDate,recruitingStartDate,requireResume,limitApplicants,limitNumber,applicants,jobQuestions,isOnline,allowOnline,jobMajors,startDate,endDate,approximateHoursPerWeek,workSchedule,URL
0,1588222,BCTR Late Night Custodian,Livestock / Night BCTR FMS,<p>Carpet cleaning and custodial work around c...,Cleaning campus buildings. Shift is 11:30 pm-2...,True,2022-09-07T05:13:39.5877101,Greg Carron,9.58,1000.0,978.0,2022-09-05T12:00:00,2021-01-01T01:00:00,True,True,10,[],[],False,False,[],NaN,NaN,NaN,NaN,https://web.byui.edu/StudentEmployment/job/158...
1,1360146,AGBUS147 OnlineTA,Campus Teaching Assistants,<p>A Teaching Assistant is needed for AGBUS 14...,A Teaching Assistant is needed for AGBUS 147: ...,True,2022-09-06T23:15:13.9391796,Esther Zonts,10.00,100.0,95.0,2022-09-14T12:00:00,2020-10-20T01:00:00,True,True,50,[],[],False,False,[],2022-06-27T08:00:00,2022-12-17T11:00:00,up to 5,Flexible,https://web.byui.edu/StudentEmployment/job/136...
2,1361237,AGBUS147 OnlineTA,Online Student,<p>A Teaching Assistant is needed for AGBUS 14...,A Teaching Assistant is needed for AGBUS 147: ...,True,2022-09-06T23:15:09.5556704,Esther Zonts,10.00,100.0,99.0,2022-09-14T12:00:00,2020-10-21T01:00:00,True,True,50,[],[],False,True,[],2022-06-26T21:00:00,2022-09-12T16:00:00,up to 5,Flexible,https://web.byui.edu/StudentEmployment/job/136...
3,2399931,Email Specialist,Main BSC Support Center,<div><div><p><span><span>As a BYU-Idaho Email ...,The BYU-Idaho Support Center is the primary su...,True,2022-09-06T22:31:55.4427994,Abigayle Ludlow,10.40,1000.0,959.0,2022-09-12T06:00:00,2021-08-10T01:00:00,True,False,0,[],[],False,False,[],2022-09-01T18:00:00,2022-09-09T18:00:00,NaN,1pm-5pm,https://web.byui.edu/StudentEmployment/job/239...
4,2182195,INTST100 OnlineTutor,Campus Student,<p><span> Online Tutor is needed for the INTST...,Online Tutor is needed for the INTST 100: Intr...,True,2022-09-06T21:31:54.3026567,Casey McDaniel,10.00,100.0,95.0,2022-09-12T23:00:00,2021-06-15T01:00:00,True,False,0,[],[],True,False,[],2022-08-09T21:00:00,2022-09-10T22:00:00,5-10 hours (must post 7 hours of appt openings...,flexible,https://web.byui.edu/StudentEmployment/job/218...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,96139,Student Project Manager,Online Curriculum Development,<p>The student project manager position undert...,Assist in managing the audiovisual section of ...,True,2022-07-29T15:12:11.4490012,Brian Carter,10.00,500.0,494.0,2022-09-09T12:00:00,2016-11-28T01:00:00,True,True,30,[],[],False,False,[],2022-07-29T12:00:00,2022-09-09T12:00:00,20,Varies,https://web.byui.edu/StudentEmployment/job/96139
338,3820655,Hard Floor Team Member,Building/Hard Floor FMS,"<p> Duties include cleaning, buffing, re-surfa...",This team member will work under the direction...,True,2022-07-25T20:17:12.8614333,Lauren Brisco,9.58,1000.0,989.0,NaN,2022-05-03T01:00:00,True,False,0,[],[],False,False,[],NaN,NaN,15,"Monday-Friday, 8PM-11PM",https://web.byui.edu/StudentEmployment/job/382...
339,32596,Chemistry Stockroom/TA/Grader,Chemistry,<p>This is a general application form for any ...,General Application form for all chemistry dep...,True,2022-06-09T20:08:58.2027174,Mike Wood,NaN,10000.0,10000.0,NaN,2016-11-11T01:00:00,True,False,0,[],[],False,False,"[{'jobMajorId': 212, 'jobId': 32596, 'majorId'...",2022-06-09T14:00:00,NaN,NaN,Varies,https://web.byui.edu/StudentEmployment/job/32596
340,341293,Security-Lock up Patrol,Public Safety,<p>Employees will assist with patrolling campu...,NaN,True,2022-02-18T23:30:52.6069815,Kevin Rhoades,NaN,200.0,135.0,NaN,2018-08-15T01:00:00,True,False,0,[],[],False,False,[],2021-12-05T07:00:00,NaN,NaN,NaN,https://web.byui.edu/StudentEmployment/job/341293


## SIMPLE OVERVIEW

In [8]:
print(f'''
    THERE ARE {data_jobs.jobID.nunique()} JOBS.
    HIGHEST PAY JOB:\'{data_jobs[data_jobs.payRate==data_jobs.payRate.max()]['title'].iloc[0]}\' WITH {data_jobs.payRate.max()} DOLLARS AN HOUR.
    ONLINE JOBS: {data_jobs[data_jobs["title"].str.contains('Online')].shape[0]} OUT OF {data_jobs.shape[0]}.
''')


    THERE ARE 342 JOBS.
    HIGHEST PAY JOB:'Stores and Receiving Delivery Driver' WITH 11.58 DOLLARS AN HOUR.
    ONLINE JOBS: 304 OUT OF 342.



## CLEAN THE DATA

In [9]:
col_dates = ['dateUpdated','startDate','endDate','beginningDate','recruitingStartDate']

data_jobs[col_dates] = data_jobs[col_dates].astype('datetime64[ns]')

data_jobs.dateUpdated = data_jobs.dateUpdated.apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

columns_to_drop = [
    'jobID', # not needed for EDA
    # 'description', # not needed for EDA
    # 'summary', # not needed for EDA
    'displayJob', # single boolean
    # 'dateUpdated',
    'startDate','endDate', # not needed for display
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    # 'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'limitApplicants', # not relevant
    'limitNumber', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]

data_jobs.drop(columns_to_drop, axis=1, inplace=True)

data_jobs['description'] = data_jobs['description'].apply(lambda x: [p.text.strip() for p in BeautifulSoup(x).find_all('p') if p.text.strip() != ''])

## OPTIONAL: Save as a CSV

In [ ]:
# data_jobs.to_csv('StudentEmployment.csv')

## FILTER OUT THE JOBS THAT HAVE 'Online','Custodian', and 'TA' IN THEIR TITLE

In [ ]:
# JOBS FILTERED. NOT ONLINE, NOR CUSTODIAN, AND NOR TA.

remove = ['Online','Custodian','TA']

data_filtered = data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False).reset_index().drop('index',axis=1)

print(f'{data_filtered.shape[0]} JOBS OUT OF {data_jobs.shape[0]} POSTED JOBS.')

pd.set_option('display.max_rows', None)

data_filtered

## OPTIONAL: Save as a CSV

In [ ]:
# data_filtered.to_csv('StudentEmployment_filtered.csv')

In [ ]:
data_jobs.payRate.value_counts().reset_index().sort_values('index')

In [ ]:
data_jobs.groupby('payRate').count()

In [ ]:
data_jobs.departmentName.value_counts()

In [ ]:
data_jobs[data_jobs.title == 'AGBUS147 OnlineTA'].iloc[0]

In [ ]:
pd.options.display.max_columns = None
data_jobs[data_jobs.jobID == 342319]

In [ ]:
data_jobs.dateUpdated.iloc[0].strftime("%Y-%m-%d")

In [17]:
import time
ym = time.strftime("%Y-%m-%d")
ym

'2022-09-07'

In [15]:
data_jobs.dateUpdated.iloc[0]

'2022-09-07'

In [20]:
time.strftime("%Y-%m-%d")

'2022-09-07'

In [23]:
from datetime import datetime, timedelta
(datetime.today() - timedelta(1)).strftime("%Y-%m-%d") 

'2022-09-06'

In [29]:
(33/1)

33.0

In [32]:
(1/33) - 1

-0.9696969696969697

In [34]:
import requests

In [35]:
response = requests.get('https://web.byui.edu/studentemployment/api/jobs')

In [36]:
soup = BeautifulSoup(response.text)

In [ ]:
soup.select()